In [1]:
from tensorflow.keras.layers import (Input, Conv2D, Flatten, MaxPooling2D,
                                     Dense, Dropout, Concatenate)
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import numpy as np
import os
import pickle

In [2]:
SUBJECT_NUM = 2
TARGET_FOLDER = "subjects/"
PATH2PROJECT = "/content/drive/MyDrive/Projects/EyeTracker/"
CHOSEN_INPUTS = [0, 1, 2, 6, 7, 8, 9]

In [3]:
x1 = []
x2 = []
y = []

in_blink_out_fol = PATH2PROJECT + TARGET_FOLDER + f"{SUBJECT_NUM}/eye_tracking data-calibration/"

with open(in_blink_out_fol + "x1.pickle", "rb") as f:
    x1 = pickle.load(f)
with open(in_blink_out_fol + "x2.pickle", "rb") as f:
    x2 = pickle.load(f)
with open(in_blink_out_fol + "y.pickle", "rb") as f:
    y = pickle.load(f)

In [4]:
x2_chs_inp = x2[:, CHOSEN_INPUTS]

In [5]:
# model5
input1 = Input(x1.shape[1:])
layer = Conv2D(16, (5, 5), (1, 1), 'same', activation='relu')(input1)
layer = MaxPooling2D((2, 2), (2, 2))(layer)

layer = Conv2D(32, (5, 5), (1, 1), 'same', activation='relu')(layer)
layer = MaxPooling2D((2, 2), (2, 2))(layer)

layer = Conv2D(64, (3, 3), (1, 1), activation='relu')(layer)
layer = MaxPooling2D((2, 2), (2, 2))(layer)

layer = Flatten()(layer)

layer = Dense(400, 'relu')(layer)

input2 = Input(x2_chs_inp.shape[1:])
layer = Concatenate()([layer, input2])

layer = Dense(180, 'relu')(layer)

layer = Dense(50, 'relu')(layer)

layer = Dense(16, 'relu')(layer)

layer = Dense(2, 'relu')(layer)

out = Dense(1, 'linear')(layer)

input_layers = [input1, input2]

model = Model(inputs=input_layers, outputs=out)

model.compile(optimizer='adam', loss='mae')

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 44, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 48, 44, 16)   416         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 24, 22, 16)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 24, 22, 32)   12832       max_pooling2d[0][0]              
______________________________________________________________________________________________

In [6]:
# plot_model(model, show_shapes=True)

In [7]:
models_numbers = []
models_dir = PATH2PROJECT + "models/eye_tracking/raw/"
models_name = os.listdir(models_dir)
for model_name in models_name:
  model_num = int(model_name[5:6])
  models_numbers.append(model_num)

max_num = max(models_numbers)

model.save(models_dir + f"model{max_num+1}_x")
model.save(models_dir + f"model{max_num+1}_y")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Projects/EyeTracker/models/eye_tracking/raw/model4_x/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Projects/EyeTracker/models/eye_tracking/raw/model4_y/assets
